In [3]:
import os
import shutil 
# 這兩個套件是為了要將訓練資料的路徑讀取工具也引進來

import tensorflow as tf # 就是 Tensorflow 不用多說
import tensorflow_hub as hub # Tensorflow bert 社群
import tensorflow_text as text # 前處理
from official.nlp import optimization  # 優化工具

import matplotlib.pyplot as plt # 畫圖用的套件

tf.get_logger().setLevel('ERROR') # Debug的工具，也可以不寫

In [4]:
# # 這個是資料集的下載連結，如果你複製這段 url 到瀏覽器上，也可以下載的到
# url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'

# dataset = tf.keras.utils.get_file('aclImdb_v1.tar.gz', url,
#                                   untar=True, cache_dir='.',
#                                   cache_subdir='')

# # 所謂的 os.path.join 就是把兩個路徑結合起來的意思，如果路徑是 ./data/，join之後就會變成 ./data/aclImdb，而以下所做的，就是要整合資料路徑，方便之後模型讀取
# dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

# train_dir = os.path.join(dataset_dir, 'train')

# # remove unused folders to make it easier to load the data
# remove_dir = os.path.join(train_dir, 'unsup')
# shutil.rmtree(remove_dir)


In [13]:
from core.general import *
ls('../data')

import pandas as pd
f = pd.read_csv('../data/train_data.csv')
f = f[['Statement', 'tags']]
f['isDP'] = ['dp' in i for i in f['tags'].tolist()]
f = f.drop('tags', axis = 1)
f

,Statement,isDP
0,After watching the new over-rated series Squid...,False
1,Today on a math lesson the teacher told Vovoch...,False
2,"A sequence of integers $$$b_1, b_2, \ldots, b_...",True
3,Petya got interested in grammar on his third y...,False
4,"Nezzar designs a brand new game ""Hidden Permut...",False
...,...,...
7971,"You are given an array of integers $$$a_1,a_2,...",True
7972,Polycarp has a poor memory. Each day he can re...,False
7973,This is the hard version of the problem. The o...,False
7974,"Two chess pieces, a rook and a knight, stand o...",False


In [6]:
pre = f.to_dict()
# for i in train:
# debug(pwd())
cd(root)
cd('aclImdb/train')
for tag in ['pos', True], ['neg', False]:
    cd(tag[0])
    for i in range(len(f['Statement'])):
        if pre['isDP'][i] ^ tag[1]: continue;
        write_to_file(f'{i}.txt', pre['Statement'][i])
    cd('..')
cd('../..')

In [7]:
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

class_names = raw_train_ds.class_names
train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)

val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)

val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

test_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/test',
    batch_size=batch_size)

test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)


Found 7976 files belonging to 2 classes.
Using 6381 files for training.


2025-01-13 17:54:07.863620: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-01-13 17:54:07.863739: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2025-01-13 17:54:07.863846: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: ysh-laptop
2025-01-13 17:54:07.863894: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: ysh-laptop
2025-01-13 17:54:07.864075: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: NOT_FOUND: was unable to find libcuda.so DSO loaded into this program
2025-01-13 17:54:07.864291: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 560.35.3
2025-01-13 17:54:07.866572: I tensorflow/core/platform/cpu_feature_guard.cc:151] This Tens

Found 7976 files belonging to 2 classes.
Using 1595 files for validation.
Found 25000 files belonging to 2 classes.


In [8]:
for text_batch, label_batch in train_ds.take(1):
    print(f'Review: {text_batch.numpy()[0]}')
    for i in range(3):
        label = label_batch.numpy()[i]
        print(f'Label : {label} ({class_names[label]})')

Review: b"This is an interactive problem.Khanh has $$$n$$$ points on the Cartesian plane, denoted by $$$a_1, a_2, \\ldots, a_n$$$. All points' coordinates are integers between $$$-10^9$$$ and $$$10^9$$$, inclusive. No three points are collinear. He says that these points are vertices of a convex polygon; in other words, there exists a permutation $$$p_1, p_2, \\ldots, p_n$$$ of integers from $$$1$$$ to $$$n$$$ such that the polygon $$$a_{p_1} a_{p_2} \\ldots a_{p_n}$$$ is convex and vertices are listed in counter-clockwise order.Khanh gives you the number $$$n$$$, but hides the coordinates of his points. Your task is to guess the above permutation by asking multiple queries. In each query, you give Khanh $$$4$$$ integers $$$t$$$, $$$i$$$, $$$j$$$, $$$k$$$; where either $$$t = 1$$$ or $$$t = 2$$$; and $$$i$$$, $$$j$$$, $$$k$$$ are three distinct indices from $$$1$$$ to $$$n$$$, inclusive. In response, Khanh tells you:  if $$$t = 1$$$, the area of the triangle $$$a_ia_ja_k$$$ multiplied 

2025-01-13 17:54:09.962463: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [9]:
tfhub_handle_encoder = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1" # 這是預訓練模型
tfhub_handle_preprocess = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3" # 這是 BERT 前處理需要用的模型

In [15]:
debug(pre['Statement'][0])

DEBUG After watching the new over-rated series Squid Game, Mashtali and Soroush decided to hold their own Squid Games! Soroush agreed to be the host and will provide money for the winner's prize, and Mashtali became the Front Man!$$$m$$$ players registered to play in the games to win the great prize, but when Mashtali found out how huge the winner's prize is going to be, he decided to kill eliminate all the players so he could take the money for himself!Here is how evil Mashtali is going to eliminate players:There is an unrooted tree with $$$n$$$ vertices. Every player has $$$2$$$ special vertices $$$x_i$$$ and $$$y_i$$$.In one operation, Mashtali can choose any vertex $$$v$$$ of the tree. Then, for each remaining player $$$i$$$ he finds a vertex $$$w$$$ on the simple path from $$$x_i$$$ to $$$y_i$$$, which is the closest to $$$v$$$. If $$$w\ne x_i$$$ and $$$w\ne y_i$$$, player $$$i$$$ will be eliminated.Now Mashtali wondered: "What is the minimum number of operations I should perform 

In [16]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess) # 呼叫前處理模型
text_test = [pre['Statement'][0]] # 先輸入簡單的句子看看會變成什麼樣子
text_preprocessed = bert_preprocess_model(text_test)
print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Keys       : ['input_word_ids', 'input_mask', 'input_type_ids']
Shape      : (1, 128)
Word Ids   : [  101  2044  3666  1996  2047  2058  1011  6758  2186 26852  2208  1010]
Input Mask : [1 1 1 1 1 1 1 1 1 1 1 1]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


In [20]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)
debug('---')
bert_results = bert_model(text_preprocessed)
print(f'Loaded BERT: {tfhub_handle_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')


DEBUG ---
Loaded BERT: https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Pooled Outputs Shape:(1, 512)
Pooled Outputs Values:[ 0.8727445   0.917182   -0.26135486  0.17491198 -0.22153176 -0.8765021
  0.99814    -0.97380465 -0.74489594 -0.9697282  -0.29953644 -0.9590922 ]
Sequence Outputs Shape:(1, 128, 512)
Sequence Outputs Values:[[-0.46256995  0.22983482 -0.15339413 ... -0.04324936  0.8477006
  -0.06155542]
 [-0.8721482  -0.26003942 -0.4240116  ... -0.01380096 -0.506868
   0.80026275]
 [-0.9195258   1.058085    1.0833865  ...  0.5954932   0.53631127
   1.3289824 ]
 ...
 [-1.0381106  -0.08092016  0.25179073 ...  0.22761185  0.11991769
  -0.3399005 ]
 [-1.3327754   0.16316739 -0.94576204 ... -0.04032091  0.2967144
  -0.76678807]
 [-0.5673314  -0.11084812 -0.6279895  ... -0.30947086  0.49631402
  -0.37789452]]


In [21]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)


In [22]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

epochs = 5
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')


In [25]:
classifier_model = build_classifier_model()

In [26]:
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)
print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(x=train_ds,
                               validation_data=val_ds,
                               epochs=epochs)            

Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Epoch 1/5


200/200 [==============================] - 909s 4s/step - loss: 0.5494 - binary_accuracy: 0.7789 - val_loss: 0.5145 - val_binary_accuracy: 0.7868
Epoch 2/5
200/200 [==============================] - 842s 4s/step - loss: 0.5144 - binary_accuracy: 0.7847 - val_loss: 0.5143 - val_binary_accuracy: 0.7868
Epoch 3/5
200/200 [==============================] - 506s 3s/step - loss: 0.4791 - binary_accuracy: 0.7877 - val_loss: 0.5153 - val_binary_accuracy: 0.7868
Epoch 4/5
200/200 [==============================] - 272s 1s/step - loss: 0.4347 - binary_accuracy: 0.7945 - val_loss: 0.5245 - val_binary_accuracy: 0.7862
Epoch 5/5
200/200 [==============================] - 272s 1s/step - loss: 0.4038 - binary_accuracy: 0.8061 - val_loss: 0.5283 - val_binary_accuracy: 0.7843


In [27]:
loss, accuracy = classifier_model.evaluate(test_ds)

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

782/782 [==============================] - 313s 400ms/step - loss: 1.1744 - binary_accuracy: 0.5000
Loss: 1.1744247674942017
Accuracy: 0.5000399947166443


In [28]:
classifier_model.save_weights('model.st')